In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import guitarpro
import glob, os, json
from utils import *

## Count Stats

Some statistics regarding the playing technique instances in the dataset. 

Note-level Counts: 

| Features      | Note Count
| ----------- | -----------
| Normal      | 109957
| Tie         | 4728
| Rest        | 0
| Dead        | 388

Effect-level Counts:

| Features      | Note Count
| ----------- | -----------
| Ghost (ignore)       | 321
| Hammer (both Hammer-on and Pull-off)      | 7536
| Mute        | 14074
| Vibrato     | 2420
| Harmonic (all types)   | 589
| Bend (all types)        | 3338
| Grace (based on the type, can be slide/hammer/bend)      | 269
| Trill (ignore)       | 0
| Slide       | 2309


In [ ]:
CLEAN_SINGLE_TRACK_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_gtps"
)

normal_notes_cnt = 0
tie_notes_cnt = 0
rest_notes_cnt = 0
dead_notes_cnt = 0

for file in glob.glob(os.path.join(CLEAN_SINGLE_TRACK_DIR, "*.gp5")):
    track_title, _ = os.path.splitext(file.split("/")[-1])
    print(f"processing file: {track_title}")

    song = guitarpro.parse(file)
    # process single track GP files
    assert len(song.tracks) == 1
    for j, measure in enumerate(song.tracks[0].measures):
        for beat in measure.voices[0].beats:
            # if mono
            if len(beat.notes) == 1:
                if beat.notes[0].type.name == "normal":
                    normal_notes_cnt += 1
                if beat.notes[0].type.name == "tie":
                    tie_notes_cnt += 1
                if beat.notes[0].type.name == "rest":
                    print(f"rest note at measure {j}")
                    rest_notes_cnt += 1
                if beat.notes[0].type.name == "dead":
                    print(f"dead note at measure {j}")
                    dead_notes_cnt += 1     

print(normal_notes_cnt) # 109957
print(tie_notes_cnt)    # 4728
print(rest_notes_cnt)   # 0
print(dead_notes_cnt)   # 388

In [ ]:
CLEAN_SINGLE_TRACK_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_gtps"
)

ghost_notes_cnt = 0
hammer_cnt = 0
mute_cnt = 0
vibrato_cnt = 0
harmonic_cnt = 0
bend_cnt = 0
grace_cnt = 0
trill_cnt = 0
slide_cnt = 0

for file in glob.glob(os.path.join(CLEAN_SINGLE_TRACK_DIR, "*.gp5")):
    track_title, _ = os.path.splitext(file.split("/")[-1])
    print(f"processing file: {track_title}")

    song = guitarpro.parse(file)
    # process single track GP files
    assert len(song.tracks) == 1
    for j, measure in enumerate(song.tracks[0].measures):
        for beat in measure.voices[0].beats:
            # if mono
            if len(beat.notes) == 1:
                if beat.notes[0].effect.ghostNote:
                    print(f"ghost note at measure {j}")
                    ghost_notes_cnt += 1
                if beat.notes[0].effect.hammer:
                    hammer_cnt += 1
                if beat.notes[0].effect.palmMute:
                    print(f"mute at measure {j}")
                    mute_cnt += 1
                if beat.notes[0].effect.vibrato:
                    vibrato_cnt += 1 
                if beat.notes[0].effect.isHarmonic:
                    harmonic_cnt += 1 
                if beat.notes[0].effect.isBend:
                    bend_cnt += 1
                if beat.notes[0].effect.isGrace:
                    grace_cnt += 1 
                if beat.notes[0].effect.isTrill:
                    print(f"trill at measure {j}")
                    trill_cnt += 1 
                if beat.notes[0].effect.slides:
                    slide_cnt += 1

print(f"ghost {ghost_notes_cnt}") # 321  
# ghost notes don't sound much different from normal notes, maybe just treat them as normal notes
print(f"hammer {hammer_cnt}")    # 7536
print(f"mute {mute_cnt}")   # 14074
print(f"vibrato {vibrato_cnt}")   # 2420
print(f"harmonic {harmonic_cnt}") # 589
print(f"bend {bend_cnt}")    # 3338
print(f"grace {grace_cnt}")   # 269
print(f"trill {trill_cnt}")   # 0
# no trill, just ignore it
print(f"slide {slide_cnt}") # 2309


## Test Anno Gen with One file

In [12]:
# basic situation
# do not merge tied notes 
# do not adjust bend info
# do not calculate the segment onsets

# for every single-track GP file, 
# this should result in the same number of annotation files as mono audio segments
CLEAN_SINGLE_TRACK_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_gtps"
)
ANNO_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/test_anno"
file = os.path.join(CLEAN_SINGLE_TRACK_DIR, "Guns N' Roses - Welcome To The Jungle (ver 3)_Solo Guitar.gp5")

song = guitarpro.parse(file)
bpm = song.tempo
# process single track GP files
assert len(song.tracks) == 1

note_infos = []
segment_idx = 0

# put all beats of the song in one place
beats = []
for measure in song.tracks[0].measures:
    beats.extend(measure.voices[0].beats)

for i, beat in enumerate(beats):
    # if poly beat or the last beat, write the accumulated list to a json file
    if len(beat.notes) > 1 or i == len(beats) - 1:
        if note_infos:
            # write to file
            with open(os.path.join(ANNO_DIR, f"test_{segment_idx}.json"), "w") as outfile:
                json.dump(note_infos, outfile, indent=4)
            segment_idx += 1
            # restart accumulation
            note_infos = []
    # if mono beat, accumulate note infos in the list
    else:
        if beat.notes:
            note = beat.notes[0]
            # add note info to the list
            note_infos.append(get_note_info(note, bpm))
        # if the beat has zero note, add sth empty so that the empty annotation matches the empty mono audio segment
        else:
            note_infos.append({})

# the tied note should be treated as the same pitch as the previous note, 
# but the note in Python could have be noted differently and should be adjusted,
# some cases are: normal, tie, tie, where the latter two notes should be adjusted to follow the first note

# some slide-out instances don't sound realistic, they sounds just like the note without slide

In [9]:
# try to get things done in one go
# distribute all beats to the segments
CLEAN_SINGLE_TRACK_DIR = (
    "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/clean_single_track_gtps"
)
ANNO_DIR = "/Volumes/MacOnly/UG_proc/all_time_top_by_hits/test_anno"
file = os.path.join(CLEAN_SINGLE_TRACK_DIR, "Guns N' Roses - Welcome To The Jungle (ver 3)_Solo Guitar.gp5")
song = guitarpro.parse(file)
# process single track GP files
assert len(song.tracks) == 1

# put all beats of the song in one place
beats = []
for measure in song.tracks[0].measures:
    beats.extend(measure.voices[0].beats)

segment_idx = 0
segments = [] # a list of lists
segment = [] # a list of beat instances
for i, beat in enumerate(beats):
    # if poly beat or the last beat, add the accumulated `segment` list to `segments`
    if len(beat.notes) > 1 or i == len(beats) - 1:
        if segment:
            segments.append(segment)
            segment_idx += 1
            # restart accumulation
            segment = []
    # if mono beat, accumulate beat instances in the `segment` list
    else:
        segment.append(beat)


In [11]:
bpm = song.tempo
for i, segment in enumerate(segments):
    segment_start = segment[0].start
    segment_start_sec = round(((segment_start - 960) / 960) / (bpm / 60), 4)

    note_infos = []
    for beat in segment:
        assert len(beat.notes) < 2
        if beat.notes:
            note = beat.notes[0]
            note_info = get_note_info(note, bpm, segment_start_sec)
            # if current note is tied, add its duration to the previous note
            if note_info["type"] == "tie":
                note_infos[-1]["time"]["dur"] = note_infos[-1]["time"]["dur"] + note_info["time"]["dur"]
            else:
                note_infos.append(note_info)

    with open(os.path.join(ANNO_DIR, f"test_{i}.json"), "w") as outfile:
        json.dump(note_infos, outfile, indent=4)
